# exp23

exp16にtsfreshの時系列特徴量を追加したものに  
Nanが多い特徴量はDropした状態で実施



exp16のスコア

1Fold：0.7959229889532946  　　0.7969883338523035
2Fold：0.796323672863839       0.7954764983395108
3Fold：0.7959284796989159      0.7941681892822336
4Fold：0.7961656161300223      0.7946013318261542
5Fold：0.7957363246647742      0.79518510573899


https://www.kaggle.com/code/ragnar123/amex-lgbm-dart-cv-0-7977

In [3]:
# ====================================================
# Library
# ====================================================
import gc
import warnings
warnings.filterwarnings('ignore')
import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
import itertools


In [4]:

# ====================================================
# Get the difference
# ====================================================
def get_difference(data, num_features):
    df1 = []
    customer_ids = []
    for customer_id, df in tqdm(data.groupby(['customer_ID'])):
        # Get the differences
        diff_df1 = df[num_features].diff(1).iloc[[-1]].values.astype(np.float32)
        # Append to lists
        df1.append(diff_df1)
        customer_ids.append(customer_id)
    # Concatenate
    df1 = np.concatenate(df1, axis = 0)
    # Transform to dataframe
    df1 = pd.DataFrame(df1, columns = [col + '_diff1' for col in df[num_features].columns])
    # Add customer id
    df1['customer_ID'] = customer_ids
    return df1

# ====================================================
# Read & preprocess data and save it to disk
# ====================================================
def read_preprocess_data():
    train = pd.read_parquet('/content/data/train.parquet')
    features = train.drop(['customer_ID', 'S_2'], axis = 1).columns.to_list()
    cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68",
    ]
    num_features = [col for col in features if col not in cat_features]
    print('Starting training feature engineer...')
    train_num_agg = train.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
    train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]
    train_num_agg.reset_index(inplace = True)
    train_cat_agg = train.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
    train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns]
    train_cat_agg.reset_index(inplace = True)
    train_labels = pd.read_csv('../input/amex-default-prediction/train_labels.csv')
    # Transform float64 columns to float32
    cols = list(train_num_agg.dtypes[train_num_agg.dtypes == 'float64'].index)
    for col in tqdm(cols):
        train_num_agg[col] = train_num_agg[col].astype(np.float32)
    # Transform int64 columns to int32
    cols = list(train_cat_agg.dtypes[train_cat_agg.dtypes == 'int64'].index)
    for col in tqdm(cols):
        train_cat_agg[col] = train_cat_agg[col].astype(np.int32)
    # Get the difference
    train_diff = get_difference(train, num_features)
    train = train_num_agg.merge(train_cat_agg, how = 'inner', on = 'customer_ID').merge(train_diff, how = 'inner', on = 'customer_ID').merge(train_labels, how = 'inner', on = 'customer_ID')
    del train_num_agg, train_cat_agg, train_diff
    gc.collect()
    test = pd.read_parquet('../input/amex-fe/test_fe.parquet')
    print('Starting test feature engineer...')
    test_num_agg = test.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
    test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]
    test_num_agg.reset_index(inplace = True)
    test_cat_agg = test.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
    test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]
    test_cat_agg.reset_index(inplace = True)
    # Transform float64 columns to float32
    cols = list(test_num_agg.dtypes[test_num_agg.dtypes == 'float64'].index)
    for col in tqdm(cols):
        test_num_agg[col] = test_num_agg[col].astype(np.float32)
    # Transform int64 columns to int32
    cols = list(test_cat_agg.dtypes[test_cat_agg.dtypes == 'int64'].index)
    for col in tqdm(cols):
        test_cat_agg[col] = test_cat_agg[col].astype(np.int32)
    # Get the difference
    test_diff = get_difference(test, num_features)
    test = test_num_agg.merge(test_cat_agg, how = 'inner', on = 'customer_ID').merge(test_diff, how = 'inner', on = 'customer_ID')
    del test_num_agg, test_cat_agg, test_diff
    gc.collect()
    # Save files to disk
    train.to_parquet('../input/amex-fe/train_fe.parquet')
    test.to_parquet('../input/amex-fe/test_fe.parquet')

# Read & Preprocess Data
read_preprocess_data()

FileNotFoundError: [Errno 2] No such file or directory: '/content/data/train.parquet'

# Training & Inference

In [5]:
# ====================================================
# Library
# ====================================================
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from itertools import combinations

# ====================================================
# Configurations
# ====================================================
class CFG:
    input_dir = '../feature/exp03_amex-fe/'
    output_dir = '../output/exp19_tsfresh_lgb/'
    seed = 46
    n_folds = 5
    target = 'target'
    boosting_type = 'dart'
    metric = 'binary_logloss'
    model = "lgb"
    ver = "exp19"

# ====================================================
# Seed everything
# ====================================================
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

# ====================================================
# Read data
# ====================================================
def read_data():
    train = pd.read_parquet(CFG.input_dir + 'train_fe_plus_plus.parquet')
    test = pd.read_parquet(CFG.input_dir + 'test_fe_plus_plus.parquet')
    return train, test

# ====================================================
# Amex metric
# ====================================================
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)

# ====================================================
# LGBM amex metric
# ====================================================
def lgb_amex_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'amex_metric', amex_metric(y_true, y_pred), True


In [6]:
seed_everything(CFG.seed)
train, test = read_data()


In [9]:

tsfresh_trian = pd.read_pickle('../feature/exp18_1_tsfresh/train_partial_autocorrelation_223.pkl')
tsfresh_test = pd.read_pickle('../feature/exp18_1_tsfresh/test_partial_autocorrelation_223.pkl')

train = train.merge(tsfresh_trian,on = "customer_ID",how = "left")
test = test.merge(tsfresh_test,on = "customer_ID",how = "left")

In [10]:
train.head()

,customer_ID,P_2_first,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_first,D_39_mean,D_39_std,D_39_min,D_39_max,D_39_last,B_1_first,B_1_mean,B_1_std,B_1_min,B_1_max,B_1_last,B_2_first,B_2_mean,B_2_std,B_2_min,B_2_max,B_2_last,R_1_first,R_1_mean,R_1_std,R_1_min,R_1_max,R_1_last,S_3_first,S_3_mean,S_3_std,S_3_min,S_3_max,S_3_last,D_41_first,D_41_mean,D_41_std,D_41_min,D_41_max,D_41_last,B_3_first,B_3_mean,B_3_std,B_3_min,B_3_max,B_3_last,D_42_first,D_42_mean,D_42_std,D_42_min,D_42_max,D_42_last,D_43_first,D_43_mean,D_43_std,D_43_min,D_43_max,D_43_last,D_44_first,D_44_mean,D_44_std,D_44_min,D_44_max,D_44_last,B_4_first,B_4_mean,B_4_std,B_4_min,B_4_max,B_4_last,D_45_first,D_45_mean,D_45_std,D_45_min,D_45_max,D_45_last,B_5_first,B_5_mean,B_5_std,B_5_min,B_5_max,B_5_last,R_2_first,R_2_mean,R_2_std,R_2_min,R_2_max,R_2_last,D_46_first,D_46_mean,D_46_std,D_46_min,D_46_max,D_46_last,D_47_first,D_47_mean,D_47_std,D_47_min,D_47_max,D_47_last,D_48_first,D_48_mean,D_48_std,D_48_min,D_48_max,D_48_last,D_49_first,D_49_mean,D_49_std,D_49_min,D_49_max,D_49_last,B_6_first,B_6_mean,B_6_std,B_6_min,B_6_max,B_6_last,B_7_first,B_7_mean,B_7_std,B_7_min,B_7_max,B_7_last,B_8_first,B_8_mean,B_8_std,B_8_min,B_8_max,B_8_last,D_50_first,D_50_mean,D_50_std,D_50_min,D_50_max,D_50_last,D_51_first,D_51_mean,D_51_std,D_51_min,D_51_max,D_51_last,B_9_first,B_9_mean,B_9_std,B_9_min,B_9_max,B_9_last,R_3_first,R_3_mean,R_3_std,R_3_min,R_3_max,R_3_last,D_52_first,D_52_mean,D_52_std,D_52_min,D_52_max,D_52_last,P_3_first,P_3_mean,P_3_std,P_3_min,P_3_max,P_3_last,B_10_first,B_10_mean,B_10_std,B_10_min,B_10_max,B_10_last,D_53_first,D_53_mean,D_53_std,D_53_min,D_53_max,D_53_last,S_5_first,S_5_mean,S_5_std,S_5_min,S_5_max,S_5_last,B_11_first,B_11_mean,B_11_std,B_11_min,B_11_max,B_11_last,S_6_first,S_6_mean,S_6_std,S_6_min,S_6_max,S_6_last,D_54_first,D_54_mean,D_54_std,D_54_min,D_54_max,D_54_last,R_4_first,R_4_mean,R_4_std,R_4_min,R_4_max,R_4_last,S_7_first,S_7_mean,S_7_std,S_7_min,S_7_max,S_7_last,B_12_first,B_12_mean,B_12_std,B_12_min,B_12_max,B_12_last,S_8_first,S_8_mean,S_8_std,S_8_min,S_8_max,S_8_last,D_55_first,D_55_mean,D_55_std,D_55_min,D_55_max,D_55_last,D_56_first,D_56_mean,D_56_std,D_56_min,D_56_max,D_56_last,B_13_first,B_13_mean,B_13_std,B_13_min,B_13_max,B_13_last,R_5_first,R_5_mean,R_5_std,...,D_117_first,D_117_last,D_117_nunique,D_120_count,D_120_first,D_120_last,D_120_nunique,D_126_count,D_126_first,D_126_last,D_126_nunique,D_63_count,D_63_first,D_63_last,D_63_nunique,D_64_count,D_64_first,D_64_last,D_64_nunique,D_66_count,D_66_first,D_66_last,D_66_nunique,D_68_count,D_68_first,D_68_last,D_68_nunique,target,P_2__partial_autocorrelation__lag_1,P_2__partial_autocorrelation__lag_3,P_2__partial_autocorrelation__lag_5,D_39__partial_autocorrelation__lag_1,D_39__partial_autocorrelation__lag_3,D_39__partial_autocorrelation__lag_5,B_1__partial_autocorrelation__lag_1,B_1__partial_autocorrelation__lag_3,B_1__partial_autocorrelation__lag_5,B_2__partial_autocorrelation__lag_1,B_2__partial_autocorrelation__lag_3,B_2__partial_autocorrelation__lag_5,R_1__partial_autocorrelation__lag_1,R_1__partial_autocorrelation__lag_3,R_1__partial_autocorrelation__lag_5,S_3__partial_autocorrelation__lag_1,S_3__partial_autocorrelation__lag_3,S_3__partial_autocorrelation__lag_5,B_3__partial_autocorrelation__lag_1,B_3__partial_autocorrelation__lag_3,B_3__partial_autocorrelation__lag_5,D_43__partial_autocorrelation__lag_1,D_43__partial_autocorrelation__lag_3,D_43__partial_autocorrelation__lag_5,B_4__partial_autocorrelation__lag_1,B_4__partial_autocorrelation__lag_3,B_4__partial_autocorrelation__lag_5,D_45__partial_autocorrelation__lag_1,D_45__partial_autocorrelation__lag_3,D_45__partial_autocorrelation__lag_5,B_5__partial_autocorrelation__lag_1,B_5__partial_autocorrelation__lag_3,B_5__partial_autocorrelation__lag_5,D_46__partial_autocorrelation__lag_1,D_46__partial_autocorrelation__lag_3,D_46__partial_autocorrelation__lag_5,D_47__partial_autocorrelation__lag_1,D_47__partial_autocorrelation__

In [11]:
cat_features = [
    "B_30",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68"
]

# kmeans_list = ["kmeans pred 2","kmeans pred 3","kmeans pred 4"]

cat_features = [f"{cf}_last" for cf in cat_features]
# cat_features.extend(kmeans_list)

for cat_col in cat_features:
#     print(cat_col)
    encoder = LabelEncoder()
    train[cat_col] = encoder.fit_transform(train[cat_col])
    test[cat_col] = encoder.transform(test[cat_col])


features = [col for col in train.columns if col not in ['customer_ID', CFG.target]]
params = {
    'objective': 'binary',
    'metric': CFG.metric,
    'boosting': CFG.boosting_type,
    'seed': CFG.seed,
    'num_leaves': 100,
    'learning_rate': 0.01,
    'feature_fraction': 0.20,
    'bagging_freq': 10,
    'bagging_fraction': 0.50,
    'n_jobs': -1,
    'lambda_l2': 2,
    'min_data_in_leaf': 40,
    }
# Create a numpy array to store test predictions
test_predictions = np.zeros(len(test))
# Create a numpy array to store out of folds predictions
oof_predictions = np.zeros(len(train))

cids = []
tr_target = []

kfold = StratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = CFG.seed)
for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[CFG.target])):
    print(' ')
    print('-'*50)
    print(f'Training fold {fold} with {len(features)} features...')
    x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
    y_train, y_val = train[CFG.target].iloc[trn_ind], train[CFG.target].iloc[val_ind]
    lgb_train = lgb.Dataset(x_train, y_train, categorical_feature = cat_features)
    lgb_valid = lgb.Dataset(x_val, y_val, categorical_feature = cat_features)
    model = lgb.train(
        params = params,
        train_set = lgb_train,
        num_boost_round = 10500,#10500
        valid_sets = [lgb_train, lgb_valid],
        early_stopping_rounds = 1500,
        verbose_eval = 500,
        feval = lgb_amex_metric
        )
    
    # Save best model
    joblib.dump(model, f'{CFG.output_dir}lgbm_{CFG.boosting_type}_fold{fold}_seed{CFG.seed}.pkl')
    # Predict validation
    val_pred = model.predict(x_val)
    # Add to out of folds array
    oof_predictions[val_ind] = val_pred
    
    cids.extend(train["customer_ID"].loc[val_ind])
    tr_target.extend(train["target"].loc[val_ind])
    
    # Predict the test set
    test_pred = model.predict(test[features])
    test_predictions += test_pred / CFG.n_folds
    # Compute fold metric
    score = amex_metric(y_val, val_pred)
    print(f'Our fold {fold} CV score is {score}')
    del x_train, x_val, y_train, y_val, lgb_train, lgb_valid
    gc.collect()
    
# Compute out of folds metric
score = amex_metric(train[CFG.target], oof_predictions)
print(f'Our out of folds CV score is {score}')


# Create a dataframe to store test prediction
test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
# test_df.to_csv(f'{CFG.output_dir}test_{CFG.model}_{score}_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)
test_df.to_csv(f'{CFG.output_dir}test_{CFG.model}_{score}_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)


dic_oof = {
    "customer_ID":cids,
    "target":tr_target,
    "tabnet_oot":oof_predictions
}

# Create a dataframe to store out of folds predictions
oof_df = pd.DataFrame(dic_oof)
# oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train[CFG.target], 'prediction': oof_predictions})
oof_df.to_csv(f'{CFG.output_dir}oof_{CFG.ver}_{CFG.model}_{score}_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)


# # Create a dataframe to store out of folds predictions
# oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train[CFG.target], 'prediction': oof_predictions})
# oof_df.to_csv(f'../output/Amex LGBM Dart CV 0.7977/oof_lgbm_{CFG.boosting_type}_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)


# # Create a dataframe to store test prediction
# test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
# test_df.to_csv(f'../output/Amex LGBM Dart CV 0.7977/test_lgbm_{CFG.boosting_type}_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)


 
--------------------------------------------------
Training fold 0 with 1682 features...
[LightGBM] [Info] Number of positive: 95062, number of negative: 272068
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.985291 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 286265
[LightGBM] [Info] Number of data points in the train set: 367130, number of used features: 1673
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258933 -> initscore=-1.051523
[LightGBM] [Info] Start training from score -1.051523
[500]	training's binary_logloss: 0.309867	training's amex_metric: 0.780588	valid_1's binary_logloss: 0.312695	valid_1's amex_metric: 0.77149
[1000]	training's binary_logloss: 0.249294	training's amex_metric: 0.794651	valid_1's binary_logloss: 0.255226	valid_1's amex_metric: 0.778038
[1500]	training's binary_logloss: 0.22342	training's amex_metric: 0.807382	valid_1's binary_logloss: 0.233256	valid_1's am

KeyboardInterrupt: 